In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.notebook.scroll_to_top();
                jupyter.actions.call('jupyter-notebook:save-notebook');                
                
            });
        });
        
        $( document ).ready(function(){
        code_shown=false;
        $('div.input').hide()});
    
    
</script>

# Overview

This program is designed to calculate potential crop yields based on a time frame and location specified by the user.

It has three parts: 
* Data Download (must be done first) 
* Crop Yields at a Specific Location Over Time
* Crop Yields over a Region at a Specific Time

This file downloads climate datasets from the Famine Land Data Assimilation System (FLDAS)* operated by the U.S. National Air and Space Administration. This data can then be used to explore the water status and related soil condition in a given area over time as an indicator of crop yields.

*McNally, A., Arsenault, K., Kumar, S., Shukla, S., Peterson, P., Wang, S., Funk, C., Peters-Lidard, C.D., & Verdin, J. P. (2017). A land data assimilation system for sub-Saharan Africa food and water security applications. Scientific Data, 4, 170012*


## 2 Steps to Get a NASA Account and Download the Data

1. An Earthdata username and password are required to run this program 

>["How to" get an EarthData account](https://wiki.earthdata.nasa.gov/display/EL/How+To+Register+For+an+EarthData+Login+Profile)

2. You must also activate data access **Failure to do this will give you an error in the other scripts**

>["How To" activate the archive download](https://disc.gsfc.nasa.gov/earthdata-login)

***

\* [More information about FLDAS](https://ldas.gsfc.nasa.gov/FLDAS/)  
\* More details on Cropy Yield (under consturction)

***




## 0: The Dependencies

This cell imports the python dependencies we use to download the data. It has already run. If you want to know which dependencies this code is using click Hide/Show Code. If you click the run cell button, the buttons for this cell will disappear. 

In [2]:
#Library downloads
from toggle_code import toggle_code as hide_code
from toggle_code import run_code as run_code

import sys
import time
import json
import urllib3
import certifi
import requests
import time
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import datetime
import requests
import glob
from io import StringIO
import pandas as pd
from netCDF4 import Dataset
import numpy as np
import os
import cgi
import ipywidgets as widgets
from ipywidgets import interact

## 1: Input Location and Time
   
To get the data for the location we will create a bounding box using the Latitude (East/West) and Longitude (North/South). To make a bounding box you want to get to the minimum and maximum latitudes and longitudes and the the code will turn these into the four corners of your box. 

The Latitude and Longitude can be found at several places. For example, [Google Maps](https://www.google.com/maps) where you can right click to identify the latitude, longitude or a website like [xwebtools](https://www.xwebtools.com/find-latitude-longitude-from-address/). 

You will need four points: 
* min longitude
* max longitude
* min latitude
* max latitude

This creates a bounding box around your area of interest. 

You will also need the time of the data you would like to download. The FLDAS dataset goes back over 30 years (1987).

* start time in form yyyy-mm-dd 
* end time in form yyy-mm-dd



In [3]:
#This code is utilized to download the FLDAS datasets for climate data used in the pcse_part2 file

hide_code()
run_code()

# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

# Set the URL for the GES DISC subset service endpoint
svcurl = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL
# It is created for convenience since this task will be repeated more than once
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty request')
        sys.exit(1)
    return response

#Data inputs
product = 'FLDAS_NOAH01_C_GL_M_001'
varNames = ['SoilMoi00_10cm_tavg', 'SoilMoi10_40cm_tavg', 'Evap_tavg',
            'Tair_f_tavg', 'Qair_f_tavg', 'SWdown_f_tavg', 'Lwnet_tavg', 'Wind_f_tavg']

minlon = widgets.FloatText(
    value=-3.199,
    description='Min Longitude:',
    disabled=False
)

maxlon = widgets.FloatText(
    value=18.949,
    description='Max Longitude:',
    disabled=False
)

minlat = widgets.FloatText(
    value=11.32,
    description='Min Latitude:',
    disabled=False
)

maxlat = widgets.FloatText(
    value=23.801,
    description='Max Latitude:',
    disabled=False
)


#Example Time inputs
#begTime = '2019-01-01'
#endTime = '2020-01-01'

begtime = widgets.Text(
    value='2019-01-01',
    placeholder='Type something',
    description='Enter beginning time:',
    disabled=False
)

endtime = widgets.Text(
    value='2020-01-01',
    placeholder='Type something',
    description='Enter end time:',
    disabled=False
)
#begTime = input("Enter beginning time ex:'2019-01-01': ")
#endTime = input("Enter end time ex '2020-01-01': ")

def build_bbox(minlon,maxlon,minlat,maxlat):
    return [minlon,maxlon,minlat,maxlat]

def get_time(begtime, endtime):
    return [begtime,endtime]


bbox = interact(build_bbox, minlon=minlon, maxlon=maxlon,minlat=minlat,maxlat=maxlat)
time_list = interact(get_time, begtime=begtime, endtime=endtime)

doMean = True
#mapping and grid
interp = 'remapbil'
destGrid = 'cfsr0.5a'


interactive(children=(FloatText(value=-3.199, description='Min Longitude:'), FloatText(value=18.949, descripti…

interactive(children=(Text(value='2019-01-01', description='Enter beginning time:', placeholder='Type somethin…

## 2. Requesting the information from NASA

NASA's request process has three phases: 
1. Send the request to NASA and then they create a job
2. Get a Job number as a url. This url is where your request is located. We will use this url to retireve your data.
3. Repeat the process to get the elevation data, which is stored seperately.  

### First we send the request

In [12]:
hide_code()
run_code()
# This cell takes the provided paramters and converts into JSON request format. 
# Construct JSON WSP request for API method: subset
# This cell takes the provided paramters and converts into JSON request format. 
# Construct JSON WSP request for API method: subset
if ((maxlon.value-minlon.value)/4) * (maxlat.value - minlat.value) > 200: 
    print("Your request is too large. Please make your bounding box smaller and rerun.")
else: 
    subset_request = {
        'methodname': 'subset',
        'type': 'jsonwsp/request',
        'version': '1.0',
        'args': {
            'role'  : 'subset',
            'start' : begtime.value,
            'end'   : endtime.value,
            'box'   : [minlon.value, minlat.value, maxlon.value, maxlat.value],  
            'crop'  : True,
            'data'  : [{'datasetId': product,
                        'variable' : varNames[0]#, 
                        #'slice'    : dimSlice
                       },
                       {'datasetId': product,
                        'variable' : varNames[1]#,
                        #'slice'    : dimSlice
                       },
                       {'datasetId': product,
                        'variable' : varNames[2]
                       },
                       {'datasetId': product,
                        'variable' : varNames[3]
                       },
                       {'datasetId': product,
                        'variable' : varNames[4]
                       },
                       {'datasetId': product,
                        'variable' : varNames[5]
                       },
                       {'datasetId': product,
                        'variable' : varNames[6]
                       },
                       {'datasetId': product,
                        'variable' : varNames[7]
                       }]
        }
    }

    try: 
        # Submit the subset request to the GES DISC Server
        response = get_http_data(subset_request)

        # Report the JobID and initial status
        myJobId = response['result']['jobId']
        print('Job ID: '+myJobId)
        print('Job status: '+response['result']['Status'])
    except: 
        print("There was an error talking to the website, please wait a few seconds and run the cell again.")


Job ID: 5fe0eb9457cdd2107d61bf4e
Job status: Accepted


## 3. Verify Request is complete

If request has not completed. This cell will continue to print "Requesting......." until is complete. 

In [13]:
hide_code()
run_code()
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']: #Does this need "Pending"
    print("Requesting......")
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))

if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Requesting......
Job status: Succeeded (100% complete)
Job Finished:  Complete (FLDAS_NOAH01_C_GL_M_001)


## 4: Retrieve the hyper links where the data is stored

In [14]:
hide_code()
run_code()
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
        
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

# Retrieve a plain-text list of results in a single shot using the saved JobID
#these are all the downloadable links/data sets
result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
try:
    result.raise_for_status()
    urls = result.text.split('\n')
    for i in urls : print('\n%s' % i)
except :
    print('Request returned error code %d' % result.status_code)

Retrieved 24 out of 24 expected items

https://hydro1.gesdisc.eosdis.nasa.gov/data/FLDAS/FLDAS_NOAH01_C_GL_M.001/doc/README_FLDAS.pdf

https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAH01_C_GL_M.001%2F2019%2FFLDAS_NOAH01_C_GL_M.A201901.001.nc&FORMAT=bmM0Lw&BBOX=11.32%2C-3.199%2C23.801%2C18.949&LABEL=FLDAS_NOAH01_C_GL_M.A201901.001.nc.SUB.nc4&SHORTNAME=FLDAS_NOAH01_C_GL_M&SERVICE=L34RS_LDAS&VERSION=1.02&DATASET_VERSION=001&VARIABLES=SoilMoi00_10cm_tavg%2CSoilMoi10_40cm_tavg%2CEvap_tavg%2CTair_f_tavg%2CQair_f_tavg%2CSWdown_f_tavg%2CLwnet_tavg%2CWind_f_tavg

https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAH01_C_GL_M.001%2F2019%2FFLDAS_NOAH01_C_GL_M.A201902.001.nc&FORMAT=bmM0Lw&BBOX=11.32%2C-3.199%2C23.801%2C18.949&LABEL=FLDAS_NOAH01_C_GL_M.A201902.001.nc.SUB.nc4&SHORTNAME=FLDAS_NOAH01_C_GL_M&SERVICE=L34RS_LDAS&VERSION=1.02&DATASET_VERSION=001&VARIABLES=SoilMoi00_10cm_tavg%2CS

## 5: Download the datasets

First you need to enter your username and password. Do not hit return after entering. You just need to type in the username and password and will see them dynamically update below the fields.

In [16]:
hide_code()
run_code()

#Allows for longer description
style = {'description_width': 'initial'}
user_in = widgets.Text(
    value='username',
    placeholder='Type something',
    description='Enter Username:',
    disabled=False,
    style = style
)

pass_in= widgets.Text(
    value='password',
    placeholder='Type something',
    description='Enter Password:',
    disabled=False,
    style = style
)

def get_cred(user,passw):
    return [user, passw]

creds = interact(get_cred, user=user_in, passw=pass_in)

interactive(children=(Text(value='username', description='Enter Username:', placeholder='Type something', styl…

Now we are going to iterate through each hyperlink and download the data to a folder called data in your local drive.

In [17]:
hide_code()
run_code()# Sort the results into documents and URLs
# We only want the URLS with data, not the readmes
docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)

# Print out the documentation links, but do not download them
print('\nDocumentation:')
for item in docs : print(item['label']+': '+item['link'])
    
# Create a password manager to deal with the 401 response that is returned from
# Earthdata Login
#put in you username and passwoord here to login to GESDISC
#username = ""
#password = ""

username = user_in.value
password = pass_in.value

password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)

# Create a cookie jar for storing cookies. This is used to store and return the session cookie
# given to use by the data server
cookie_jar = CookieJar()
   
#Prevent cell from erroring on 1st run 
if username == "username":
    print("Waiting for Inputs")
else:
    #create direcotry
    os.makedirs("data", exist_ok=True) 
    
    # Install all the handlers.
    opener = urllib.request.build_opener (urllib.request.HTTPBasicAuthHandler (password_manager),urllib.request.HTTPCookieProcessor (cookie_jar))
    urllib.request.install_opener(opener)

    # Open a request for the data, and download files
    print('\nHTTP_services output: \n')
    for item in urls:
        URL = item['link'] 
        DataRequest = urllib.request.Request(URL)
        DataResponse = urllib.request.urlopen(DataRequest)

    # Print out the result
        DataBody = DataResponse.read()        
        # Save file to working directory
        try:
            filename = item['label']
            filepath = os.path.join("data", filename) 
            file_ = open(filepath, 'wb')
            file_.write(DataBody)
            file_.close()
            print (filepath, "has downloaded")
        except requests.exceptions.HTTPError as e:
             print(e)

    print('\n Finished Downloading All Files')


Documentation:
README Document: https://hydro1.gesdisc.eosdis.nasa.gov/data/FLDAS/FLDAS_NOAH01_C_GL_M.001/doc/README_FLDAS.pdf

HTTP_services output: 

data\FLDAS_NOAH01_C_GL_M.A201901.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201902.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201903.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201904.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201905.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201906.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201907.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201908.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201909.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201910.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201911.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A201912.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A202001.001.nc.SUB.nc4 has downloaded
data\FLDAS_NOAH01_C_GL_M.A20

## 6: Get elevation data.

Elevation data is necessary for the calculations in the crop yield notebooks. To get it we will again make a request via NASA but we will be going to a different dataset [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/). 

Due to the differences in how the data is managed, this series of request will take longer to process and depending on the size of your request may result in several messages to the email associated with your log in tell you the request is submitted and when it is processed.  

### Make the request

In [18]:
hide_code()
run_code()


if username == "username":
    print("Waiting for inputs.")
else: 
    api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'

    ###########################################
    #       Prep Coordinates for Request      #
    ###########################################
    #This code prepares the grid coordinates for
    # the request to AppEEARS by creating a grid of lat
    #longs and then putting each grid location in a request

    #Gets the coorindates form thepreviously downloaded data
    for file in glob.glob(r'data/*.nc4'):
        #print(file)
        data = Dataset(file, 'r')

    #Get lat longs
    lats = data.variables['Y'][:]
    lons = data.variables['X'][:]
    #Convert to a grid using numpy
    lon, lat = np.meshgrid(lons, lats)
    lat_flat = lat.flatten()
    lon_flat = lon.flatten()
    #Get total area
    reqs = len(lons)*len(lats)
    #Create a list for requests
    coordinates = []
    for i in range(reqs):
        coordinates.append({
            "id": str(i),
            "latitude": str(round(lat_flat[i],4)),
            "longitude": str(round(lon_flat[i],4))
            })

    #######################################################
    #            Make Requests                            #
    ######################################################

    response_elevation = requests.post('https://lpdaacsvc.cr.usgs.gov/appeears/api/login', auth=(user_in.value, pass_in.value))
    token_response_elevation = response_elevation.json()
    #print(token_response_elevation)
    token_elevation = token_response_elevation['token']
    head = {'Authorization': 'Bearer {}'.format(token_elevation)}
    tasks = []
    min_reqs = 0
    #Max number of requests for AppEEARs
    max_reqs =1000
    count = 1
    print("Your requests are being processind for submission.")
    while max_reqs < reqs: 
        #Loop through to ensure all coordinates collected
        if max_reqs//1000 != reqs//1000:
            for max_reqs in range(1000,reqs,1000):
                task = {
                    'task_type': "point",
                    'task_name': "elevation_point_request"+str(count),
                    'params': {
                         'dates': [
                         {
                             'startDate': begtime.value,
                             'endDate': endtime.value
                         }],
                         'layers': [{'layer': 'ASTER_GDEM_DEM', 'product': 'ASTGTM_NC.003'}],
                         'coordinates': coordinates[min_reqs:max_reqs]
                    }
                }

                task_response = requests.post('{}task'.format(api), json=task, headers=head).json()
                tasks.append(task_response)
                min_reqs += 1000
                count += 1
                print(task_response)
        #Get the reminder coordinates below max
        else: 
            task = {
                    'task_type': "point",
                    'task_name': "elevation_point_request_" +str(count),
                    'params': {
                         'dates': [
                         {
                             'startDate': begtime.value,
                             'endDate': endtime.value
                         }],
                         'layers': [{'layer': 'ASTER_GDEM_DEM', 'product': 'ASTGTM_NC.003'}],
                         'coordinates': coordinates[max_reqs:-1]
                    }
             }
            max_reqs = reqs
            task_response = requests.post('{}task'.format(api), json=task, headers=head).json()
            print(task_response)
            tasks.append(task_response)

    print("Your requests are submitted, please check you email to see when they have processed")



Your requests are being processind for submission.
{'task_id': 'a7615fe0-1e8a-487f-b154-406f356e2559', 'status': 'pending'}
{'task_id': '203cfcaf-f444-42d6-89d3-8777c7a73e0a', 'status': 'pending'}
{'task_id': '7f2a0405-1019-41b4-8459-dd2709ee89f7', 'status': 'pending'}
{'task_id': '52b02b8b-21a9-4578-82db-9f4923aa05ef', 'status': 'pending'}
{'task_id': '4484fd1f-94e0-4db2-87fb-87baf21b989d', 'status': 'pending'}
{'task_id': '74184d0b-6ef9-481b-beea-872776768193', 'status': 'pending'}
{'task_id': 'd1a48343-3f6b-48b1-8b4e-d3f72d389218', 'status': 'pending'}
{'task_id': '31350013-d32f-4bd7-9c16-e3d961145fca', 'status': 'pending'}
{'task_id': '25c756f1-0268-4fb7-8f4b-ef52aa33b6e0', 'status': 'pending'}
{'task_id': 'e8300c0d-63a2-4169-86b1-28dc9dd9a0b9', 'status': 'pending'}
{'task_id': 'e775ec86-9d17-4cd6-899d-0cc39373ff3f', 'status': 'pending'}
{'task_id': 'aeaa465c-1eea-4e26-8b87-5c4fbcc600b2', 'status': 'pending'}
{'task_id': '5fefdeb7-556e-47ef-ab07-43acff852e1d', 'status': 'pending'}


### Retrieve Complete Requests---It may take awhile before requests are complete.

In [20]:
hide_code()
run_code()

if username == "username":
    print("Waiting for inputs.")
else:
    print("This will provide an update every minute on the status of your requests.This may take hours")
    #Get request status
    for task in tasks: 
        starttime = time.time()
        task_id = task['task_id']
        while requests.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
                print(task_id, "is.....")
                print(requests.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
                time.sleep(60.0 - ((time.time() - starttime) % 60.0))
            
    
    #Once requests complete this retrieves the elevation data.  
    elevation_data = []
    for task in tasks: 
        task_id = task['task_id'] 
        bundle = requests.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json

        files = {}                                                       # Create empty dictionary
        for f in bundle['files']: 
            if f["file_type"] == "csv":
                files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
        for f in files: 
             dl = requests.get('{}bundle/{}/{}'.format(api, task_id, f),stream=True)     
        data = dl.content
        s=str(data,'utf-8')
        data = StringIO(s) 
        df=pd.read_csv(data)
        elevation_data.append(df)
    all_elevation = pd.concat(elevation_data)
    all_elevation = all_elevation[["Latitude", "Longitude", 'ASTGTM_NC_003_ASTER_GDEM_DEM']]
    all_elevation = all_elevation.rename(columns={'ASTGTM_NC_003_ASTER_GDEM_DEM':"Elevation"})
    file = os.path.join("data", "elevation.csv") 
    all_elevation.to_csv(file)
    print("The elevation data has been saved as a elevation.csv in you data directory.")
    del username
    del password
    del password_manager
    del creds
    user_in.value = "username"
    pass_in.value = "password"
  

This will provide an update every minute on the status of your requests.
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
This will provide an update every minute on the status of your requests.
processing
processing
processing
This will provide an update every minute on the status of your requests.


## 7. Delete Credentials Double Check

If you did not run the entire notebook (i.e. through elevation--previous cell) make sure you run the next cell to delete all your credentials.


In [22]:
hide_code()
run_code()

#Rerun deletion up above

if user_in.value == "username" and pass_in.value == "password":
    pass
    print("Username and Password not detected.")
else: 
    del username
    del password
    del password_manager
    del creds
    user_in.value = "username"
    pass_in.value = "password"
    print("Credentials Deleted")

Username and Password not detected.
